**Q5.** Use the available variables to construct a model that predicts delays.

In [40]:
#Import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, roc_auc_score, recall_score, accuracy_score

In [31]:
#Read data
data_2006 = pd.read_csv("2006.csv.bz2")
data_2007 = pd.read_csv("2007.csv.bz2")

In [32]:
#Cleanse data
data_2006 = data_2006[(data_2006['Cancelled'] == 0) & (data_2006['Diverted'] == 0)]
data_2007 = data_2007[(data_2007['Cancelled'] == 0) & (data_2007['Diverted'] == 0)]
print(data_2006.shape)
print(data_2007.shape)

(7003802, 29)
(7275288, 29)


In [33]:
#Format the data further
data_2007 = data_2007.sample(7003802)
datamerged = pd.concat([data_2006, data_2007],ignore_index=True) 

datamerged['TotDelay']= (datamerged['ArrDelay']+datamerged['DepDelay'])
datamerged['CRSDepTime']=(datamerged['CRSDepTime']/100)
datamerged['CRSArrTime']=(datamerged['CRSArrTime']/100)

#Creating classes:
Delay =[]
for row in datamerged['TotDelay']:
    if row > 30:
        Delay.append(2)    
    elif row < 30:  
        Delay.append(1)    
    else:
        Delay.append(0)    
datamerged['Delay'] = Delay 
datamerged

Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0         2006      1          11          3    743.0        7.45   1024.0   
1         2006      1          11          3   1053.0       10.53   1313.0   
2         2006      1          11          3   1915.0       19.15   2110.0   
3         2006      1          11          3   1753.0       17.55   1925.0   
4         2006      1          11          3    824.0        8.32   1015.0   
...        ...    ...         ...        ...      ...         ...      ...   
14007599  2007      7          13          5   1850.0       17.25   2028.0   
14007600  2007      8          24          5    814.0        8.15    921.0   
14007601  2007      6           4          1   1519.0       15.15   1712.0   
14007602  2007      8           3          5    815.0        8.12    830.0   
14007603  2007      7          16          1   1933.0       19.10   2118.0   

          CRSArrTime UniqueCarrier  FlightNum  ... Cancelled  \
0              10.18            US        343  ...         0   
1              13.18            US        613  ...         0   
2              21.33            US        617  ...         0   
3              19.33            US        300  ...         0   
4              10.15            US        765  ...         0   
...              ...           ...        ...  ...       ...   
14007599       18.40            MQ       3581  ...         0   
14007600        9.20            MQ       3389  ...         0   
14007601       16.52            OH       5460  ...         0   
14007602        8.37            XE       7705  ...         0   
14007603       21.00            WN        234  ...         0   

          CancellationCode  Diverted  CarrierDelay  WeatherDelay  NASDelay  \
0                      NaN         0             0             0         0   
1                      NaN         0             0             0         0   
2                      NaN         0             0             0         0   
3                      NaN         0             0             0         0   
4                      NaN         0             0             0         0   
...                    ...       ...           ...           ...       ...   
14007599               NaN         0            18             0        23   
14007600               NaN         0             0             0         0   
14007601               NaN         0             4             0        16   
14007602               NaN         0             0             0         0   
14007603               NaN         0             0             0         0   

         SecurityDelay LateAircraftDelay  TotDelay  Delay  
0                    0                 0       4.0      1  
1                    0                 0      -5.0      1  
2                    0                 0     -23.0      1  
3                    0                 0     -10.0      1  
4                    0                 0      -8.0      1  
...                ...               ...       ...    ...  
14007599             0                67     193.0      2  
14007600             0                 0       0.0      1  
14007601             0                 0      24.0      1  
14007602             0                 0      -4.0      1  
14007603             0                18      41.0      2  

[14007604 rows x 31 columns]

In [ ]:
#Select fields and View nulls
datamerged = datamerged[['Month','DayOfWeek','DayofMonth','Origin','Dest','CRSDepTime','DepDelay','CRSArrTime','ArrDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','Delay']]
datamerged.dropna(how="all", axis=1, inplace=True)
datamerged.isna().sum()

In [36]:
#Sample data and manipulate the fields
datamerged=datamerged.sample(250000)
#Creating dummy variables for Origin and Dest:
datamerged = pd.get_dummies(datamerged, columns=['Origin', 'Dest'])
datamerged.head()

Month  DayOfWeek  DayofMonth  CRSDepTime  DepDelay  CRSArrTime  \
1941706      4          2          18        9.30      12.0       12.47   
5109870      9          6          16        8.15      69.0        9.25   
1118676      3          7          26       13.55      -2.0       15.16   
1224734      3          3          15       21.35      -4.0       23.25   
6720569     12          5           8       19.42      18.0       22.02   

         ArrDelay  CarrierDelay  WeatherDelay  NASDelay  ...  Dest_TYR  \
1941706       7.0             0             0         0  ...         0   
5109870      59.0            59             0         0  ...         0   
1118676       3.0             0             0         0  ...         0   
1224734     -21.0             0             0         0  ...         0   
6720569      33.0            18             0        15  ...         0   

         Dest_TYS  Dest_VLD  Dest_VPS  Dest_WRG  Dest_WYS  Dest_XNA  Dest_YAK  \
1941706         0         0         0         0         0         0         0   
5109870         0         0         0         0         0         0         0   
1118676         0         0         0         0         0         0         0   
1224734         0         0         0         0         0         0         0   
6720569         0         0         0         0         0         0         0   

         Dest_YKM  Dest_YUM  
1941706         0         0  
5109870         0         0  
1118676         0         0  
1224734         0         0  
6720569         0         0  

[5 rows x 618 columns]

In [37]:
#Line9
#Splitting into triain (80%) and test (20%)
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(datamerged.drop('Delay', axis=1), datamerged['Delay'], test_size=0.2, random_state=42)

In [38]:
#Assigning first model, Gradient Bosting Classifier
modelgradboost = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)

probabilitygradboost = modelgradboost.predict_proba(X_test)
trainpredgradboost = modelgradboost.predict(X_train)
predictiongradboost = modelgradboost.predict(X_test)

print('Prediction: {:.5f}'.format(modelgradboost.score(X_test, y_test)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test, predictiongradboost)))
print('AUC score: {:.5f}'.format(roc_auc_score(y_test, probabilitygradboost, multi_class='ovr')))
print('Precision: {:.5f}'.format(precision_score(y_train, trainpredgradboost, average='micro')))
print('Recall: {:.5f}'.format(recall_score(y_test, predictiongradboost, average='micro')))

Prediction: 0.92464
Accuracy: 0.92464
AUC score: 0.75306
Precision: 0.92570
Recall: 0.92464


In [41]:
#Assigning second model, Decision Tree Classifier
modeltreeclass = DecisionTreeClassifier(random_state=42)
modeltreeclass.fit(X_train, y_train)

probabilitytreeclass = modeltreeclass.predict_proba(X_test)
trainpredtreeclass = modeltreeclass.predict(X_train)
predictiontreeclass = modeltreeclass.predict(X_test)

print('Prediction: {:.5f}'.format(modeltreeclass.score(X_test, y_test)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test, predictiontreeclass)))
print('AUC score: {:.5f}'.format(roc_auc_score(y_test, probabilitytreeclass, multi_class='ovr')))
print('Precision: {:.5f}'.format(precision_score(y_train, trainpredtreeclass, average='micro')))
print('Recall: {:.5f}'.format(recall_score(y_test, predictiontreeclass, average='micro')))


Prediction: 0.99914
Accuracy: 0.99914
AUC score: 0.98164
Precision: 1.00000
Recall: 0.99914
